# Práctica 4: Anemometría Láser Doppler (LDA) - Perfil de Velocidades

## Objetivo
Medir perfiles de velocidad en un túnel de viento utilizando técnica láser Doppler (LDA/LDV), una técnica óptica no intrusiva que permite obtener mediciones puntuales de velocidad con alta precisión.

## Fundamento Teórico

### ¿Qué es LDA?
La **Anemometría Láser Doppler** se basa en el efecto Doppler de la luz dispersada por partículas trazadoras que se mueven con el fluido. Dos haces láser coherentes se cruzan formando un **volumen de medida** donde se genera un patrón de franjas de interferencia.

Cuando una partícula atraviesa este volumen:
- Dispersa luz con una frecuencia modulada proporcional a su velocidad
- La frecuencia Doppler $f_D$ está relacionada con la velocidad por: 
$$V = \frac{\lambda \cdot f_D}{2 \sin(\theta/2)}$$
donde $\lambda$ es la longitud de onda del láser y $\theta$ el ángulo entre haces.

### Ventajas de LDA
- ✅ **No intrusiva**: no perturba el flujo
- ✅ **Alta resolución espacial y temporal**
- ✅ **Medición directa de velocidad** (no requiere calibración)
- ✅ **Puede medir flujos inversos** y turbulencia

### Datos LDA típicos
Cada medición registra:
1. **Tiempo de llegada** (ms)
2. **Velocidad** (m/s o componente U, V)
3. **SNR** (Signal-to-Noise Ratio) - calidad de señal
4. **Validación** - indicadores de calidad

---

## Estructura de los Datos

En `files/P4/` tenemos 4 carpetas (FX01G00 a FX04G00), cada una representa una **posición Y diferente** en el perfil del túnel.

Dentro de cada carpeta hay ~15 archivos `.txt`, cada uno con ~2000 mediciones de velocidad en esa posición.

**Formato de columnas** (sin encabezado):
1. Índice de muestra
2. Velocidad U (m/s) - componente principal
3. Algo relacionado con tiempo o frecuencia
4. Velocidad V (m/s) - componente transversal 
5. Otra componente o validación

Nuestro objetivo: **construir el perfil U(y)** promediando todas las mediciones por posición.

In [3]:
# Imports necesarios
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ Librerías cargadas correctamente")

ModuleNotFoundError: No module named 'pandas'

## 1. Configuración de Rutas y Exploración de Datos

Primero localizamos los datos en `files/P4/` y exploramos la estructura.

In [ ]:
# Configuración de rutas
# Los datos están en files/P4/ (3 niveles arriba del notebook)
BASE_DIR = Path.cwd().parent.parent.parent
DATA_DIR = BASE_DIR / 'files' / 'P4'

print(f"📂 Directorio base: {BASE_DIR}")
print(f"📂 Directorio de datos: {DATA_DIR}")
print(f"✅ Datos existen: {DATA_DIR.exists()}")

# Listar las carpetas (cada una es una posición Y)
folders = sorted([f for f in DATA_DIR.iterdir() if f.is_dir()])
print(f"\n📁 Carpetas encontradas ({len(folders)}):")
for folder in folders:
    files_count = len(list(folder.glob('*.txt')))
    print(f"  - {folder.name}: {files_count} archivos")

## 2. Lectura y Análisis de un Archivo Individual

Examinemos un archivo para entender el formato de datos.

In [ ]:
# Leer un archivo de ejemplo
sample_file = folders[0] / list(folders[0].glob('*.txt'))[0]
print(f"📄 Archivo de ejemplo: {sample_file.name}")

# Leer sin encabezado, separado por espacios
df_sample = pd.read_csv(sample_file, sep=r'\s+', header=None, 
                        names=['idx', 'U', 'col3', 'V', 'col5'])

print(f"\n📊 Forma de datos: {df_sample.shape}")
print(f"   → {df_sample.shape[0]} mediciones × {df_sample.shape[1]} columnas")

print("\n🔍 Primeras 10 filas:")
print(df_sample.head(10))

print("\n📈 Estadísticas descriptivas:")
print(df_sample.describe())

In [ ]:
# Visualización de datos brutos de un archivo
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Velocidad U
axes[0, 0].hist(df_sample['U'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Velocidad U (m/s)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Velocidad U')
axes[0, 0].axvline(df_sample['U'].mean(), color='r', linestyle='--', 
                    label=f'Media = {df_sample["U"].mean():.2f} m/s')
axes[0, 0].legend()

# Velocidad V
axes[0, 1].hist(df_sample['V'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Velocidad V (m/s)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Velocidad V')
axes[0, 1].axvline(df_sample['V'].mean(), color='r', linestyle='--',
                    label=f'Media = {df_sample["V"].mean():.3f} m/s')
axes[0, 1].legend()

# Serie temporal U
axes[1, 0].plot(df_sample['idx'], df_sample['U'], linewidth=0.5, alpha=0.7)
axes[1, 0].set_xlabel('Índice de muestra')
axes[1, 0].set_ylabel('U (m/s)')
axes[1, 0].set_title('Serie Temporal - Velocidad U')

# Box plot
axes[1, 1].boxplot([df_sample['U'], df_sample['V']], labels=['U', 'V'])
axes[1, 1].set_ylabel('Velocidad (m/s)')
axes[1, 1].set_title('Box Plot de Velocidades')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Análisis del archivo {sample_file.name}:")
print(f"   U media: {df_sample['U'].mean():.3f} m/s")
print(f"   U std:   {df_sample['U'].std():.3f} m/s")
print(f"   V media: {df_sample['V'].mean():.3f} m/s (transversal)")
print(f"   V std:   {df_sample['V'].std():.3f} m/s")

## 3. Función para Procesar Todos los Archivos de una Posición

Crearemos una función que:
1. Lee todos los archivos de una carpeta (posición Y)
2. Filtra outliers usando criterio IQR (Rango Intercuartílico)
3. Calcula estadísticos robustos (media, std, error estándar)

**Método de detección de outliers:**
- Calculamos Q1 (percentil 25) y Q3 (percentil 75)
- IQR = Q3 - Q1
- Outliers: valores fuera de [Q1 - 1.5×IQR, Q3 + 1.5×IQR]

In [ ]:
def remove_outliers_iqr(data, factor=1.5):
    """
    Elimina outliers usando el método del rango intercuartílico (IQR).
    
    Parámetros:
    -----------
    data : array-like
        Datos a filtrar
    factor : float
        Factor multiplicador del IQR (típicamente 1.5)
    
    Retorna:
    --------
    data_clean : array
        Datos sin outliers
    mask : array booleano
        Máscara de valores válidos
    """
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    
    mask = (data >= lower_bound) & (data <= upper_bound)
    return data[mask], mask


def process_position_folder(folder_path, remove_outliers=True):
    """
    Procesa todos los archivos de una carpeta (posición Y).
    
    Parámetros:
    -----------
    folder_path : Path
        Ruta a la carpeta con archivos .txt
    remove_outliers : bool
        Si True, elimina outliers con método IQR
    
    Retorna:
    --------
    results : dict
        Diccionario con estadísticos de velocidad U y V
    """
    all_U = []
    all_V = []
    
    # Leer todos los archivos .txt de la carpeta
    txt_files = list(folder_path.glob('*.txt'))
    
    for file in txt_files:
        df = pd.read_csv(file, sep=r'\s+', header=None,
                        names=['idx', 'U', 'col3', 'V', 'col5'])
        all_U.extend(df['U'].values)
        all_V.extend(df['V'].values)
    
    all_U = np.array(all_U)
    all_V = np.array(all_V)
    
    # Eliminar outliers si se solicita
    if remove_outliers:
        U_clean, mask_U = remove_outliers_iqr(all_U)
        V_clean, mask_V = remove_outliers_iqr(all_V)
        outliers_removed_U = len(all_U) - len(U_clean)
        outliers_removed_V = len(all_V) - len(V_clean)
    else:
        U_clean = all_U
        V_clean = all_V
        outliers_removed_U = 0
        outliers_removed_V = 0
    
    # Calcular estadísticos
    results = {
        'folder': folder_path.name,
        'n_files': len(txt_files),
        'n_samples_total': len(all_U),
        'n_samples_clean_U': len(U_clean),
        'n_samples_clean_V': len(V_clean),
        'outliers_removed_U': outliers_removed_U,
        'outliers_removed_V': outliers_removed_V,
        'U_mean': np.mean(U_clean),
        'U_std': np.std(U_clean),
        'U_stderr': stats.sem(U_clean),  # Error estándar de la media
        'V_mean': np.mean(V_clean),
        'V_std': np.std(V_clean),
        'V_stderr': stats.sem(V_clean),
        'U_all': all_U,
        'V_all': all_V,
        'U_clean': U_clean,
        'V_clean': V_clean
    }
    
    return results

print("✅ Funciones de procesamiento definidas")

## 4. Procesar Todas las Posiciones

Ahora procesamos las 4 carpetas (posiciones Y) y almacenamos los resultados.

In [ ]:
# Procesar todas las carpetas
results_all = []

print("🔄 Procesando posiciones...")
print("="*70)

for folder in folders:
    print(f"\n📁 Procesando: {folder.name}")
    result = process_position_folder(folder, remove_outliers=True)
    results_all.append(result)
    
    print(f"   Archivos: {result['n_files']}")
    print(f"   Muestras totales: {result['n_samples_total']}")
    print(f"   Outliers eliminados (U): {result['outliers_removed_U']} "
          f"({100*result['outliers_removed_U']/result['n_samples_total']:.1f}%)")
    print(f"   U = {result['U_mean']:.3f} ± {result['U_std']:.3f} m/s")
    print(f"   V = {result['V_mean']:.4f} ± {result['V_std']:.4f} m/s")

print("\n" + "="*70)
print("✅ Procesamiento completado")

## 5. Construcción del Perfil de Velocidades U(y)

Para graficar el perfil necesitamos asignar posiciones Y a cada carpeta. 

**Asumimos** que las carpetas están ordenadas de abajo hacia arriba (o viceversa) en el túnel. Si conoces las posiciones reales en mm, ajusta el array `y_positions`.

In [ ]:
# Posiciones Y: ajusta estos valores según tus mediciones reales
# Por ahora usamos posiciones equiespaciadas como ejemplo
y_positions = np.array([10, 30, 50, 70])  # mm, ejemplo

# Extraer datos para el perfil
U_means = np.array([r['U_mean'] for r in results_all])
U_stderrs = np.array([r['U_stderr'] for r in results_all])

# Crear DataFrame resumen
df_profile = pd.DataFrame({
    'Posición': [r['folder'] for r in results_all],
    'y (mm)': y_positions,
    'U_mean (m/s)': U_means,
    'U_std (m/s)': [r['U_std'] for r in results_all],
    'U_stderr (m/s)': U_stderrs,
    'V_mean (m/s)': [r['V_mean'] for r in results_all],
    'N_muestras': [r['n_samples_clean_U'] for r in results_all]
})

print("📊 Tabla Resumen del Perfil de Velocidades:")
print("="*80)
print(df_profile.to_string(index=False))
print("="*80)

In [ ]:
# Gráfico del perfil de velocidades U(y)
fig, ax = plt.subplots(figsize=(10, 8))

# Perfil con barras de error (error estándar)
ax.errorbar(U_means, y_positions, xerr=U_stderrs, 
            fmt='o-', markersize=10, linewidth=2, capsize=5,
            label='U(y) ± SE', color='steelblue', elinewidth=2)

# Línea de referencia de velocidad media global
U_global_mean = U_means.mean()
ax.axvline(U_global_mean, color='red', linestyle='--', linewidth=1.5,
           label=f'U media global = {U_global_mean:.2f} m/s', alpha=0.7)

ax.set_xlabel('Velocidad U (m/s)', fontsize=13, fontweight='bold')
ax.set_ylabel('Posición Y (mm)', fontsize=13, fontweight='bold')
ax.set_title('Perfil de Velocidades - Anemometría Láser Doppler (LDA)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

print(f"\n📈 Velocidad media del perfil: {U_global_mean:.3f} m/s")
print(f"   Rango de velocidades: {U_means.min():.3f} - {U_means.max():.3f} m/s")
print(f"   Variación: {U_means.max() - U_means.min():.3f} m/s")

## 6. Análisis de Turbulencia

Calculamos la **intensidad de turbulencia** en cada posición:

$$I_u = \frac{\sigma_U}{U_{mean}} \times 100\%$$

Donde:
- $\sigma_U$ es la desviación estándar de U
- $U_{mean}$ es la velocidad media

La intensidad de turbulencia indica qué tan "agitado" está el flujo respecto a su velocidad media.

In [ ]:
# Calcular intensidad de turbulencia
turbulence_intensity = []

for result in results_all:
    I_u = (result['U_std'] / result['U_mean']) * 100
    turbulence_intensity.append(I_u)

df_profile['Turbulencia (%)'] = turbulence_intensity

print("🌪️  Intensidad de Turbulencia por Posición:")
print("="*60)
for i, row in df_profile.iterrows():
    print(f"{row['Posición']:12s} (y={row['y (mm)']:5.1f} mm): "
          f"I_u = {row['Turbulencia (%)']:5.2f}%")
print("="*60)

# Gráfico de intensidad de turbulencia
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(turbulence_intensity, y_positions, 'o-', markersize=10, 
        linewidth=2, color='coral', label='Intensidad de Turbulencia')

ax.set_xlabel('Intensidad de Turbulencia I_u (%)', fontsize=13, fontweight='bold')
ax.set_ylabel('Posición Y (mm)', fontsize=13, fontweight='bold')
ax.set_title('Perfil de Intensidad de Turbulencia', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

## 7. Comparación de Distribuciones por Posición

Visualizamos las distribuciones de velocidad U en cada posición para identificar comportamientos.

In [ ]:
# Histogramas superpuestos
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

colors = ['steelblue', 'coral', 'forestgreen', 'purple']

for i, result in enumerate(results_all):
    ax = axes[i]
    
    # Histograma de datos limpios
    ax.hist(result['U_clean'], bins=50, alpha=0.7, color=colors[i], 
            edgecolor='black', label=f"Limpio (n={len(result['U_clean'])})")
    
    # Línea vertical en la media
    ax.axvline(result['U_mean'], color='red', linestyle='--', linewidth=2,
               label=f"Media = {result['U_mean']:.2f} m/s")
    
    ax.set_xlabel('Velocidad U (m/s)', fontsize=11)
    ax.set_ylabel('Frecuencia', fontsize=11)
    ax.set_title(f"{result['folder']} - y={y_positions[i]} mm", 
                 fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Análisis de Calidad de Datos

Evaluamos la calidad de las mediciones:
1. **Ratio señal/ruido implícito**: menor dispersión = mejor calidad
2. **Consistencia entre archivos**: comparar varianza inter-archivo vs intra-archivo
3. **Convergencia estadística**: verificar que tenemos suficientes muestras

In [ ]:
# Box plot comparativo de todas las posiciones
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Box plot de velocidad U
U_data = [r['U_clean'] for r in results_all]
positions_labels = [f"{r['folder']}\ny={y_positions[i]} mm" 
                   for i, r in enumerate(results_all)]

bp1 = ax1.boxplot(U_data, labels=positions_labels, patch_artist=True)
for patch, color in zip(bp1['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax1.set_ylabel('Velocidad U (m/s)', fontsize=12, fontweight='bold')
ax1.set_xlabel('Posición', fontsize=12, fontweight='bold')
ax1.set_title('Distribución de Velocidad U por Posición', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Coeficiente de variación (CV) por posición
cv_values = [(r['U_std']/r['U_mean'])*100 for r in results_all]
ax2.bar(range(len(cv_values)), cv_values, color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Posición', fontsize=12, fontweight='bold')
ax2.set_ylabel('Coeficiente de Variación (%)', fontsize=12, fontweight='bold')
ax2.set_title('Coeficiente de Variación (CV = σ/μ × 100%)', fontsize=13, fontweight='bold')
ax2.set_xticks(range(len(results_all)))
ax2.set_xticklabels([r['folder'] for r in results_all], rotation=45)
ax2.grid(True, alpha=0.3, axis='y')

# Añadir línea de referencia de calidad
ax2.axhline(10, color='red', linestyle='--', linewidth=2, 
            label='CV=10% (límite calidad)', alpha=0.7)
ax2.legend()

plt.tight_layout()
plt.show()

print("\n📊 Métricas de Calidad:")
print("="*70)
for i, (result, cv) in enumerate(zip(results_all, cv_values)):
    quality = "Excelente" if cv < 5 else "Buena" if cv < 10 else "Regular"
    print(f"{result['folder']:12s}: CV={cv:5.2f}% → Calidad: {quality}")
print("="*70)

## 9. Ajuste de Perfil Teórico (Opcional)

Para flujo en túnel, podemos comparar con perfiles teóricos:
- **Flujo laminar**: perfil parabólico (Poiseuille)
- **Flujo turbulento**: perfil logarítmico (ley de la pared) o ley de potencia

Aquí probamos un ajuste polinómico simple como aproximación.

In [ ]:
# Ajuste polinómico de orden 2 (parabólico)
from numpy.polynomial import polynomial as P

# Ajustar polinomio
coeffs = np.polyfit(y_positions, U_means, 2)
poly_fit = np.poly1d(coeffs)

# Generar puntos para curva suave
y_fine = np.linspace(y_positions.min(), y_positions.max(), 100)
U_fit = poly_fit(y_fine)

# Gráfico con ajuste
fig, ax = plt.subplots(figsize=(11, 8))

# Datos experimentales
ax.errorbar(U_means, y_positions, xerr=U_stderrs, 
            fmt='o', markersize=12, linewidth=2, capsize=6,
            label='Datos experimentales LDA', color='steelblue', 
            elinewidth=2, capthick=2)

# Ajuste polinómico
ax.plot(U_fit, y_fine, 'r--', linewidth=2.5, 
        label=f'Ajuste parabólico: U = {coeffs[0]:.4f}y² + {coeffs[1]:.4f}y + {coeffs[2]:.3f}',
        alpha=0.8)

ax.set_xlabel('Velocidad U (m/s)', fontsize=13, fontweight='bold')
ax.set_ylabel('Posición Y (mm)', fontsize=13, fontweight='bold')
ax.set_title('Perfil de Velocidades con Ajuste Teórico', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=10, loc='best')

plt.tight_layout()
plt.show()

# Calcular R² (bondad de ajuste)
U_fit_points = poly_fit(y_positions)
ss_res = np.sum((U_means - U_fit_points)**2)
ss_tot = np.sum((U_means - U_means.mean())**2)
r_squared = 1 - (ss_res / ss_tot)

print(f"\n📐 Ajuste Polinómico (orden 2):")
print(f"   Ecuación: U(y) = {coeffs[0]:.6f}·y² + {coeffs[1]:.6f}·y + {coeffs[2]:.4f}")
print(f"   R² = {r_squared:.4f}")
print(f"   {'✅ Excelente ajuste' if r_squared > 0.95 else '⚠️  Ajuste moderado'}")

## 10. Exportar Resultados

Guardamos los resultados procesados en formato CSV para análisis posterior o inclusión en informes.

In [ ]:
# Directorio de salida (en data de la práctica)
OUTPUT_DIR = Path.cwd().parent / 'data'
OUTPUT_DIR.mkdir(exist_ok=True)

# Guardar tabla de perfil
output_file = OUTPUT_DIR / 'perfil_velocidades_LDA.csv'
df_profile.to_csv(output_file, index=False, encoding='utf-8')
print(f"✅ Perfil guardado en: {output_file}")

# Guardar datos completos de cada posición (opcional, solo si necesario)
for i, result in enumerate(results_all):
    data_full = pd.DataFrame({
        'U': result['U_clean'],
        'V': result['V_clean']
    })
    file_position = OUTPUT_DIR / f"datos_completos_{result['folder']}.csv"
    data_full.to_csv(file_position, index=False)
    print(f"   → Datos de {result['folder']} guardados")

print(f"\n📁 Archivos generados en: {OUTPUT_DIR}")

## 11. Conclusiones y Resumen

### Resultados Principales

**Perfil de Velocidades U(y):**

In [ ]:
# Resumen final de resultados
print("="*80)
print(" "*20 + "📊 RESUMEN FINAL - PRÁCTICA 4 LDA")
print("="*80)
print(f"\n🎯 Objetivos cumplidos:")
print("   ✅ Procesamiento de {:.0f} mediciones totales".format(
    sum([r['n_samples_total'] for r in results_all])))
print("   ✅ Filtrado de outliers IQR aplicado")
print("   ✅ Perfil U(y) construido con 4 posiciones")
print("   ✅ Análisis de turbulencia completado")
print("   ✅ Ajuste teórico realizado")

print(f"\n📈 Estadísticas Globales:")
print(f"   Velocidad media global: {U_global_mean:.3f} m/s")
print(f"   Rango de velocidades: {U_means.min():.3f} - {U_means.max():.3f} m/s")
print(f"   Turbulencia media: {np.mean(turbulence_intensity):.2f}%")
print(f"   Turbulencia máxima: {np.max(turbulence_intensity):.2f}% "
      f"en {results_all[np.argmax(turbulence_intensity)]['folder']}")

print(f"\n🔬 Calidad de Mediciones:")
for i, result in enumerate(results_all):
    print(f"   {result['folder']:12s}: {result['n_samples_clean_U']:6d} muestras válidas, "
          f"CV={cv_values[i]:5.2f}%")

print(f"\n📐 Ajuste de Perfil:")
print(f"   Modelo: Polinómico orden 2 (parabólico)")
print(f"   R² = {r_squared:.4f}")

print(f"\n💡 Observaciones:")
if r_squared > 0.95:
    print("   • El perfil se ajusta bien a una forma parabólica")
    print("   • Sugiere flujo con características laminares o en desarrollo")
elif r_squared > 0.85:
    print("   • El perfil muestra cierta curvatura pero con desviaciones")
    print("   • Posible transición laminar-turbulento")
else:
    print("   • El perfil no es claramente parabólico")
    print("   • Considerar perfil logarítmico para flujo turbulento desarrollado")

print(f"\n🌪️  Turbulencia:")
if np.mean(turbulence_intensity) < 5:
    print("   • Nivel de turbulencia bajo → Flujo relativamente estable")
elif np.mean(turbulence_intensity) < 15:
    print("   • Nivel de turbulencia moderado → Flujo típico de túnel")
else:
    print("   • Nivel de turbulencia alto → Flujo muy perturbado")

print("\n" + "="*80)
print("✅ Análisis completado exitosamente")
print("="*80)

---

## 📚 Referencias y Siguientes Pasos

### Conceptos Clave Aplicados
1. **Anemometría Láser Doppler (LDA)**: técnica óptica no intrusiva basada en efecto Doppler
2. **Filtrado de outliers**: método IQR (Rango Intercuartílico) para eliminar mediciones espurias
3. **Perfil de velocidades**: caracterización U(y) del flujo en túnel
4. **Intensidad de turbulencia**: $I_u = \sigma_U / U_{mean} \times 100\%$
5. **Ajuste de modelos**: comparación experimental vs teórico (parabólico, logarítmico)

### Posibles Extensiones
- 🔬 Calcular número de Reynolds y clasificar régimen de flujo
- 📊 Análisis espectral (FFT) si hay información temporal detallada
- 🌀 Estimar escala de longitud de Kolmogorov para turbulencia
- 📈 Comparar con perfiles de ley de la pared ($u^+ = f(y^+)$) si flujo turbulento
- 🔄 Correlacionar U-V para tensiones de Reynolds

### Recomendaciones para el Informe
1. Incluir tabla resumen (ya exportada en CSV)
2. Gráficos principales: perfil U(y) con barras de error, distribuciones, turbulencia
3. Discutir calidad de datos (CV, outliers)
4. Comparar con teoría de flujo en canal/túnel
5. Estimar incertidumbres (error estándar ya calculado)

### Archivos Generados
- `perfil_velocidades_LDA.csv`: tabla resumen con estadísticos por posición
- `datos_completos_FXxxGxx.csv`: datos completos filtrados por carpeta

---

**Autor:** Práctica 4 - LDA  
**Fecha:** Octubre 2025  
**Herramientas:** Python, NumPy, Pandas, Matplotlib, SciPy